#### Purpose of the script

Metadata can be very useful in ChatLLM projects. One of the most common ways to extract metadata is to look at the `file_path` column on the feature group section and try to find metadata that can be extracted directly from there.

This script utilises LLM's to do just that. The process is as follows:
1. Loads a FG that is of type "documentset"
2. Finds some discreet examples of file paths
3. Provides a sample SQL code that can be used to extract useful metadata from the file_path.

Please note that the model might not do a perfect job in extracting useful metadata. Human sensibility can help iterate and make the response better, or you might just choose to directly alter the SQL that is being generated. Regardless, it can provide a good starting point for extracting metadata from file_paths.

In [ ]:
FG_TABLE = 'YOUR_DOCUMENTS_FEATURE_GROUP'

In [ ]:
import abacusai
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
client = abacusai.ApiClient()

df = client.describe_feature_group_by_table_name(FG_TABLE).load_as_pandas()

def get_diverse_samples(df, n_samples=100):
    # Convert file paths to a more normalized form and remove the filename
    paths = df['file_path'].apply(lambda x: x.lower().replace('\\', '/'))
    # Get just the directory paths by removing the last component
    dir_paths = paths.apply(lambda x: '/'.join(x.split('/')[:-1]))
    
    # Create TF-IDF vectors from the directory paths
    vectorizer = TfidfVectorizer(
        analyzer='char',
        ngram_range=(3, 3),  # Use character trigrams
        max_features=2000
    )
    
    path_vectors = vectorizer.fit_transform(dir_paths)
    
    # Determine number of clusters
    n_clusters = min(n_samples, len(df))
    
    # Perform clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(path_vectors)
    
    # Create a dictionary to store samples from each cluster
    samples = []
    
    # For each cluster, select one random sample
    for cluster_id in range(n_clusters):
        cluster_mask = clusters == cluster_id
        cluster_indices = np.where(cluster_mask)[0]
        
        if len(cluster_indices) > 0:
            # Randomly select one sample from this cluster
            selected_idx = np.random.choice(cluster_indices)
            samples.append(df.iloc[selected_idx])
    
    # Convert to DataFrame
    result_df = pd.DataFrame(samples)
    
    # If we need more samples to reach n_samples, add random ones
    if len(result_df) < n_samples:
        remaining = n_samples - len(result_df)
        additional = df.sample(n=remaining)
        
        # Instead of using drop_duplicates, we'll use index-based deduplication
        combined_indices = pd.Index(result_df.index).union(additional.index)
        result_df = df.loc[combined_indices]
        
        # If we still need more samples after deduplication
        if len(result_df) < n_samples:
            more_samples = df.drop(result_df.index).sample(n=n_samples - len(result_df))
            result_df = pd.concat([result_df, more_samples])
    
    return result_df

# Example usage:
diverse_samples = get_diverse_samples(df, n_samples=100)

In [ ]:
system_message = """
Here are the revised behavior instructions:

---

**Objective:**  
Your task is to assist in crafting a SQL query to extract metadata from file paths. This metadata will support a RAG-based application by identifying patterns that enhance our approach. We aim to extract metadata that is meaningful but not overly detailed, focusing primarily on the document type.

**Instructions:**  
1. **Analyze File Paths:**  
   Review the provided example file paths to determine the most relevant metadata to extract. The goal is to identify patterns that can inform the RAG application.

2. **Create SQL Query:**  
   Develop a SQL query that effectively extracts the necessary metadata. Use the example below as a guide, but adapt it based on the file paths you receive. The query should be logical and straightforward, avoiding unnecessary complexity.

3. **Example SQL Query:**  

```
   SELECT *, 
          CASE WHEN UPPER(split(file_path, '/')[4]) LIKE 'GENERAL SERVICES' THEN UPPER(split(file_path, '/')[5])
               WHEN UPPER(split(file_path, '/')[4]) LIKE 'SPECIFIC SERVICES' THEN UPPER(split(file_path, '/')[5])
               ELSE UPPER(split(file_path, '/')[4])
          END AS services_type,
          UPPER(split(file_path, '/')[3]) AS document_type
   FROM TABLE
```
4. **Adaptation:** 
    In the example, the query extracts metadata based on specific conditions. You may find that a simpler approach is sufficient. Consider creating 3-4 metadata columns if necessary, ensuring they provide valuable insights without being overly granular.

5. **Be Sensible:**
    Use your judgment to determine the most effective way to extract metadata. The complexity of the query should match the complexity of the file paths and the needs of the application. 
    If a metadata column has the same values across the board, then we don't care about it. Furthermore, avoid creating metadata columns that would extract the final documents name as metadata. We don't want that at all.
"""
r = client.evaluate_prompt(prompt = str(diverse_samples['file_path'].unique()), system_message = system_message)
print(r.content)